In [1]:
import pandas as pd

df = pd.read_csv('s3://daeggu-bucket/dk.csv'
                 ,index_col=0
                 ,names=['customer_id', 'product_id', 'star_rating']
                 ,delimiter=',',error_bad_lines=False)
df.head(16)
df.to_csv('dk_input.csv', sep='\t')

In [ ]:
import pandas as pd

df = pd.read_csv('s3://daeggu-bucket/train/dk_input.csv', delimiter='\t',error_bad_lines=False)
df.head()

In [ ]:
bucket = 'daeggu-bucket'
train = 'train'
output= 'output'

import sagemaker
role = sagemaker.get_execution_role()

import os
import mxnet as mx
from mxnet import gluon, nd, ndarray
from mxnet.metric import MSE
import pandas as pd
import numpy as np
import sagemaker
from sagemaker.mxnet import MXNet
import boto3
import json
import matplotlib.pyplot as plt

opt = 'sgd'
lr = 0.02
momentum = 0.9
wd = 0.

#submit
m = MXNet('recommender.py', 
          py_version='py3',
          role=role, 
          train_instance_count=1, 
          train_instance_type="ml.p2.xlarge",
          output_path='s3://{}/{}'.format(bucket, output),
          hyperparameters={'num_embeddings': 512, 
                           'opt': opt, 
                           'lr': lr, 
                           'momentum': momentum, 
                           'wd': wd,
                           'epochs': 10})

m.fit({'train': 's3://{}/{}/'.format(bucket, train)})

In [ ]:
# #DEPLOY
predictor = m.deploy(initial_instance_count=1, 
                     instance_type='ml.m4.xlarge')
predictor.serializer = None

In [ ]:
predictor.predict(json.dumps({'customer_id': ["user_id"], 
                              'product_id': [1,2,3,4,5]}))